In [1]:
import pandas as pd
import numpy as np
from cataclop.core import models
from cataclop.core.managers import queryset_iterator
from django.forms.models import model_to_dict

In [2]:
players = models.Player.objects.filter(is_racing=True)
races = models.Race.objects.all()
sessions = models.RaceSession.objects.all()
hippos = models.Hippodrome.objects.all()

In [4]:
players = pd.DataFrame.from_records(players.values(), index='id')
races = pd.DataFrame.from_records(races.values(), index='id')
sessions = pd.DataFrame.from_records(sessions.values(), index='id')
hippos = pd.DataFrame.from_records(hippos.values(), index='id')

In [6]:
for c in ['category', 'condition_age', 'condition_sex', 'sub_category']:
    races[c] = races[c].astype('category')
    
for c in ['country']:
    hippos[c] = hippos[c].astype('category')


In [12]:
hippos.index.name = "hippodrome_id"
sessions.index.name = "session_id"
races.index.name = "race_id"

In [13]:
s2 = sessions.join(hippos, on="hippodrome_id", lsuffix="_session", rsuffix="_hippo")
r2 = races.join(s2, on="session_id", lsuffix="_race", rsuffix="_session")
p2 = players.join(r2, on="race_id", lsuffix="_player", rsuffix="_race")

p2.reset_index(inplace=True)
p2['start_at_date'] = p2['start_at'].dt.date
p2.set_index(['start_at_date', 'race_id', 'id'], inplace=True)

Index(['age', 'earnings', 'final_odds_ref', 'handicap_distance',
       'handicap_weight', 'herder_id', 'horse_id', 'imported_at_player',
       'is_first_timer', 'is_racing', 'jockey_id', 'num', 'owner_id',
       'placed_2_count', 'placed_3_count', 'placed_count', 'placed_dividend',
       'placed_earnings', 'position', 'post_position', 'prev_year_earnings',
       'race_count', 'race_id', 'time', 'trainer_id', 'victory_count',
       'victory_earnings', 'winner_dividend', 'year_earnings', 'category',
       'condition_age', 'condition_sex', 'declared_player_count', 'distance',
       'imported_at_race', 'num_race', 'num_bis', 'prize', 'session_id',
       'start_at', 'sub_category', 'date', 'hippodrome_id',
       'imported_at_session', 'num_session', 'code', 'country',
       'imported_at_hippo', 'name'],
      dtype='object')


In [30]:
p2.loc(axis=0)[:][['category', 'num_session', 'num_race', 'country', 'name', 'position', 'final_odds_ref', 'winner_dividend']][0:10]
p2[['winner_dividend', 'placed_dividend']] = p2[['winner_dividend', 'placed_dividend']].fillna(0.0)

In [31]:
races = p2.groupby('race_id')

In [45]:
import random

bets = []

for (id, race) in races:
    
    bet = 1
    
    num = random.randint(0,min(len(race)-1, 3))
    player = race.iloc[num]
    
    profit = player['winner_dividend']/100.0 * bet - bet
    
    bets.append( (id, bet, player['num'], profit) )
    
bets = pd.DataFrame(bets, columns=['id', 'bet', 'num', 'profit'])

bets['stash'] = bets['profit'].cumsum()

bets


,id,bet,num,profit,pool
0,1,1,2,-1.0,-1.0
1,8,1,1,-1.0,-2.0
2,9,1,3,-1.0,-3.0
3,10,1,2,3.1,0.1
4,11,1,1,-1.0,-0.9
5,12,1,3,8.5,7.6
6,14,1,4,-1.0,6.6
7,16,1,3,-1.0,5.6
8,18,1,2,-1.0,4.6
9,21,1,3,-1.0,3.6
